In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
!tar xf spark-3.2.4-bin-hadoop2.7.tgz
!pip install -q findspark==1.3.0

In [2]:
!pip install -q pyspark==3.2.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 18.8 MB/s eta 0:00:00


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"

In [4]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

findspark.init("spark-3.2.4-bin-hadoop2.7")
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('abc').getOrCreate()

In [6]:
from pyspark.sql.functions import split, explode, count, lower, length
import time
import urllib.request


Opis zadania

Znajdź 5 książek o objętości ponad 200 stron każda i włącz w jeden zbiór danych.

Zamień znaki na małe. Policz znaki występujące w książce{ a: 100, ą : 150,.... } (1p) Wykonaj obliczenia za pomocą RDD oraz Df. (2p) Porównaj czasy wykonania obliczeń. (1p)

Poeksperymentuj, jak zmienią się czasy jeżeli napiszesz coś nieco inaczej, np. zmienisz jakiś krok. Co się stanie jak zbiór wzrośnie do 10, 30 książek, jak zmienią się czasy obliczeń? (3p)
Zadanie należy udostępnić mi w formie pliku colab. Arkusz powinien zawierać wszystkie wykonane kroki (również te związane z instalacją).

Rozwiazanie

In [7]:
x = ["Dark-Desire.txt", "Mansfield-Park.txt", "Pride-and-Prejudice.txt", "Sense-and-Sensibility.txt", "The-Secret-of-Poppyridge-Cove.txt"]
url_git = "https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/"

ksiazki = []

for i in x:
    url = url_git + i
    print(url)
    with urllib.request.urlopen(url) as f:
        ksiazka = f.read().decode()
        ksiazki.append(ksiazka)


https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Dark-Desire.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Mansfield-Park.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Pride-and-Prejudice.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Sense-and-Sensibility.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Secret-of-Poppyridge-Cove.txt


In [9]:
#RDD
#mozemy zauwazyc ze czas wykonania tego zadania za pomoca RDD trwa srednio od 5 sekund 
start= time.time()
for i, ksiazka in enumerate(ksiazki):
    znaki = sc.parallelize(ksiazka.lower().split())
    liczba = znaki.flatMap(lambda x: list(x)).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).collect()
    print(f"{x[i]} {liczba}")

end = time.time()
print(f"Uplynelo {end-start} sekund") 

Dark-Desire.txt [('l', 7782), ('y', 4482), ('b', 3270), ('s', 10929), ('d', 10647), ('0', 12), ('c', 3969), ('r', 9509), ('1', 40), ('g', 3491), ('.', 3659), ('i', 13668), ('4', 8), ('8', 8), ('9', 7), ('p', 3357), ('“', 804), ('h', 12065), ('j', 325), ('?', 389), ('!', 63), (';', 108), ('>', 4), ('…', 56), ('ï', 1), ('$', 1), ('|', 2), ('o', 15215), ('v', 1546), ('k', 2564), ('t', 18881), ('2', 13), ('-', 77), ('w', 5678), ('e', 24641), ('n', 12973), ('a', 15840), ('7', 8), ('u', 5911), ('m', 5910), ('3', 9), ('5', 8), ('6', 7), (',', 3604), ('’', 1918), ('”', 806), ('f', 3813), ('z', 96), ('q', 121), ('x', 426), ('<', 4), ("'", 9), ('é', 5), ('/', 6), ('ç', 2), (':', 8), ('—', 5), ('‘', 4), ('%', 1), ('(', 7), (')', 7), ('&', 1), ('@', 1)]
Mansfield-Park.txt [('h', 43786), ('p', 11329), ('r', 43928), ('j', 1007), ('c', 16962), ('g', 14128), ('b', 11116), ('s', 44817), ('i', 47284), ('l', 26364), ('d', 30166), ('y', 15509), ('0', 28), ('1', 75), ('[', 2), ('#', 1), ('4', 18), ('8', 12

In [10]:
#DF
#mozemy zauwazyc ze czas wykonania tego samego zadania ale za pomoca DF jest o wiele mniejszy i trwa srednio od 2 sekund
start= time.time()

df = spark.createDataFrame([(x[i], ksiazki[i]) for i in range(len(x))], ["nazwa_ksiazki", "tekst"])
df = df.withColumn("maly_tekst", lower(df["tekst"])) 
df = df.withColumn("znak", explode(split(df["maly_tekst"], ""))) 
#df = df.filter(df["char"].isin(list("abcdefghijklmnopqrstuvwxyz"))) #mozemy wyszukac tylko litery 
df = df.filter(~df["znak"].isin(list(" \t\n"))) #usuwam niepotrzebne znaki typu  \t\n i spacje
df = df.where(length(df["znak"]) > 0) #pozostawiam znaki ktore maja dlug wiecej niz 0
df = df.groupBy("nazwa_ksiazki", "znak").agg(count("*").alias("liczba_znakow")) 

df.show()

end = time.time()
print(f"Uplynelo {end-start} sekund")


+------------------+----+-------------+
|     nazwa_ksiazki|znak|liczba_znakow|
+------------------+----+-------------+
|   Dark-Desire.txt|   6|            7|
|Mansfield-Park.txt|   s|        44817|
|   Dark-Desire.txt|   y|         4482|
|Mansfield-Park.txt|   4|           18|
|   Dark-Desire.txt|   q|          121|
|Mansfield-Park.txt|   ?|          471|
|Mansfield-Park.txt|   u|        20937|
|   Dark-Desire.txt|   t|        18881|
|   Dark-Desire.txt|   u|         5911|
|   Dark-Desire.txt|   o|        15215|
|Mansfield-Park.txt|   @|            2|
|Mansfield-Park.txt|   y|        15509|
|Mansfield-Park.txt|   i|        47284|
|   Dark-Desire.txt|   ”|          806|
|   Dark-Desire.txt|   ç|            2|
|   Dark-Desire.txt|   7|            8|
|Mansfield-Park.txt|   :|          360|
|   Dark-Desire.txt|   i|        13668|
|Mansfield-Park.txt|   d|        30166|
|   Dark-Desire.txt|   9|            7|
+------------------+----+-------------+
only showing top 20 rows

Uplynelo 11.63

In [11]:

#ten kod pokazuje tylko wybrane litery, a nie wszystkie znaki za pomoc fukcji .filter() i .isin()
#jak mozemy zauwazyc sredni czas wykonanaia tego zadania wariuje od 3 sekund
start= time.time()
df= spark.createDataFrame([(x[i], ksiazki[i]) for i in range(len(x))], ["nazwa_ksiazki", "tekst"])
df = df.withColumn("maly_tekst", lower(df["tekst"])) 
df = df.withColumn("znak", explode(split(df["maly_tekst"], ""))) 
df= df.filter(df["znak"].isin(list("abcdefghijklmnopqrstuvwxyz"))) 
df = df.filter(~df["znak"].isin(list(" \t\n"))) #usuwam niepotrzebne znaki typu  \t\n i spacje
df = df.where(length(df["znak"]) > 0) #pozostawiam znaki ktore maja dlug wiecej niz 0
df = df.groupBy("nazwa_ksiazki", "znak").agg(count("*").alias("liczba_znakow")).orderBy("znak")
df.show()

end = time.time()
print(f"Uplynelo {end-start} sekund")

+--------------------+----+-------------+
|       nazwa_ksiazki|znak|liczba_znakow|
+--------------------+----+-------------+
|Sense-and-Sensibi...|   a|        41525|
|  Mansfield-Park.txt|   a|        56281|
|The-Secret-of-Pop...|   a|        17918|
|     Dark-Desire.txt|   a|        15840|
|Pride-and-Prejudi...|   a|        42741|
|  Mansfield-Park.txt|   b|        11116|
|The-Secret-of-Pop...|   b|         4331|
|     Dark-Desire.txt|   b|         3270|
|Sense-and-Sensibi...|   b|         8197|
|Pride-and-Prejudi...|   b|         9357|
|     Dark-Desire.txt|   c|         3969|
|Pride-and-Prejudi...|   c|        14114|
|The-Secret-of-Pop...|   c|         5515|
|Sense-and-Sensibi...|   c|        13131|
|  Mansfield-Park.txt|   c|        16962|
|  Mansfield-Park.txt|   d|        30166|
|Pride-and-Prejudi...|   d|        22843|
|     Dark-Desire.txt|   d|        10647|
|The-Secret-of-Pop...|   d|        11186|
|Sense-and-Sensibi...|   d|        22840|
+--------------------+----+-------

In [12]:
#DF2
#przeksztalcilam wierszy w kolumny za pomoca .pivot() f-kcji, jak mozemy zauwazyc, to sie zwiekszyl czas wykonania danego zadania od 4 sekund zamiast od 2 sekund jak w poprzednim
start= time.time()

df= spark.createDataFrame([(x[i], ksiazki[i]) for i in range(len(x))], ["nazwa_ksiazki", "tekst"])
df = df.withColumn("maly_tekst", lower(df["tekst"])) 
df = df.withColumn("znak", explode(split(df["maly_tekst"], ""))) 

#df= df.filter(df["znak"].isin(list("abcdefghijklmnopqrstuvwxyz")))  #jesli chce zeby pokazzywaly tylko litery
df = df.filter(~df["znak"].isin(list(" \t\n"))) #usuwam niepotrzebne znaki typu  \t\n i spacje
df = df.where(length(df["znak"]) > 0) #pozostawiam znaki ktore maja dlug wiecej niz 0
df = df.groupBy("nazwa_ksiazki").pivot("znak").agg(count("*").alias("liczba_znakow")) 

df.show()

end = time.time()
print(f"Uplynelo {end-start} sekund")

+--------------------+---+----+----+---+----+----+----+---+---+----+-----+---+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+---+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+----+-----+----+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+
|       nazwa_ksiazki|  !|   "|   #|  $|   %|   &|   '|  (|  )|   *|    ,|  -|   .|  /|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|  :|   ;|   <|   >|  ?|   @|   [|   ]|    a|    b|    c|    d|    e|    f|    g|    h|    i|   j|   k|    l|    m|    n|    o|    p|  q|    r|    s|    t|    u|   v|    w|   x|    y|  z|   ||   £|   à|   ç|   é|   ê|   ï|   —|   ‘|   ’|   “|   ”|   …|
+--------------------+---+----+----+---+----+----+----+---+---+----+-----+---+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+---+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+-----+-----+-

In [13]:

#10 ksiazek
x10 = ["Dark-Desire.txt", "Mansfield-Park.txt", "Pride-and-Prejudice.txt", "Sense-and-Sensibility.txt", "The-Secret-of-Poppyridge-Cove.txt", "Alien-Cradle.txt", "From-the-Earth-to-the-Moon.txt", "The-Cat-of-Bubastes.txt", "The-Restless-Sex.txt", "The-Window-at-the-White-Cat.txt"]
url_git10 = "https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/"

ksiazki10 = []
import urllib.request
for i in x10:
    url10 = url_git10 + i
    print(url10)
    with urllib.request.urlopen(url10) as f:
        ksiazka10 = f.read().decode()
        ksiazki10.append(ksiazka10)

https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Dark-Desire.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Mansfield-Park.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Pride-and-Prejudice.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Sense-and-Sensibility.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Secret-of-Poppyridge-Cove.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Alien-Cradle.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/From-the-Earth-to-the-Moon.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Cat-of-Bubastes.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Restless-Sex.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Window-at-the-White-Cat.txt


In [14]:
#jak mozemy zauwazyc sredni czas wykonania danego zadania z 10 ksiazkami zwiekszyl sie prawie w dwa razy porownujac do zadania z 5 ksiazkami
start= time.time()
for i, ksiazka10 in enumerate(ksiazki10):
    znaki10 = sc.parallelize(ksiazka10.lower().split())
    liczba10 = znaki10.flatMap(lambda x: list(x)).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).collect()
    print(f"{x10[i]} {liczba}")

end = time.time()
print(f"Uplynelo {end-start} sekund")

Dark-Desire.txt [('h', 16290), ('s', 14116), ('c', 5515), ('r', 12001), ('p', 4061), ('y', 4649), ('i', 14824), ('d', 11186), ('g', 5545), ('l', 9312), ('1', 37), ('4', 7), ('8', 8), ('9', 7), ('0', 24), ('!', 67), ('b', 4331), ('.', 4225), ('j', 373), ('…', 98), ('*', 31), ('“', 1062), ('?', 400), (';', 6), ('à', 1), ('$', 2), ('#', 6), ('ï', 1), ('t', 19229), ('e', 30042), ('o', 15276), ('f', 4329), ('v', 1811), ('m', 4651), ('a', 17918), ('n', 14978), ('u', 6376), ('2', 15), ('3', 12), ('5', 9), ('6', 8), ('7', 6), (',', 3128), ('w', 5442), ('-', 106), ('k', 2571), ('’', 1586), ('z', 219), ('(', 5), (')', 5), (':', 14), ('/', 1), ('”', 1062), ('q', 231), ('—', 101), ('x', 280), ("'", 16), ('é', 3), ('‘', 1)]
Mansfield-Park.txt [('h', 16290), ('s', 14116), ('c', 5515), ('r', 12001), ('p', 4061), ('y', 4649), ('i', 14824), ('d', 11186), ('g', 5545), ('l', 9312), ('1', 37), ('4', 7), ('8', 8), ('9', 7), ('0', 24), ('!', 67), ('b', 4331), ('.', 4225), ('j', 373), ('…', 98), ('*', 31), (

In [15]:
#jak mozemy zauwazyc sredni czas wykonania danego zadania z 10 ksiazkami zwiekszyl sie prawie w dwa razy porownujac do zadania z 5 ksiazkami

start= time.time()

df10 = spark.createDataFrame([(x10[i], ksiazki10[i]) for i in range(len(x10))], ["nazwa_ksiazki", "tekst"])
df10 = df10.withColumn("maly_tekst", lower(df10["tekst"])) 
df10 = df10.withColumn("znak", explode(split(df10["maly_tekst"], ""))) 
#df = df.filter(df["char"].isin(list("abcdefghijklmnopqrstuvwxyz"))) 
df10 = df10.filter(~df10["znak"].isin(list(" \t\n"))) #usuwam niepotrzebne znaki typu  \t\n i spacje
df10 = df10.where(length(df10["znak"]) > 0) #pozostawiam znaki ktore maja dlug wiecej niz 0
df10 = df10.groupBy("nazwa_ksiazki", "znak").agg(count("*").alias("liczba_znakow")) 

df10.show(10)

end = time.time()
print(f"Uplynelo {end-start} sekund")

+--------------------+----+-------------+
|       nazwa_ksiazki|znak|liczba_znakow|
+--------------------+----+-------------+
|     Dark-Desire.txt|   6|            7|
|  Mansfield-Park.txt|   s|        44817|
|The-Secret-of-Pop...|   1|           37|
|The-Secret-of-Pop...|   ’|         1586|
|     Dark-Desire.txt|   y|         4482|
|  Mansfield-Park.txt|   4|           18|
|Sense-and-Sensibi...|   g|         9949|
|     Dark-Desire.txt|   q|          121|
|  Mansfield-Park.txt|   ?|          471|
|The-Secret-of-Pop...|   ‘|            1|
+--------------------+----+-------------+
only showing top 10 rows

Uplynelo 5.068523168563843 sekund


In [16]:
#jak mozemy zauwazyc sredni czas wykonania danego zadania z 10 ksiazkami zwiekszyl sie prawie w dwa razy porownujac do zadania z 5 ksiazkami
start= time.time()


df10 = spark.createDataFrame([(x10[i], ksiazki10[i]) for i in range(len(x10))], ["nazwa_ksiazki", "tekst"])
df10 = df10.withColumn("maly_tekst", lower(df10["tekst"])) 
df10 = df10.withColumn("znak", explode(split(df10["maly_tekst"], ""))) 
#df = df.filter(df["znak"].isin(list("abcdefghijklmnopqrstuvwxyz"))) 
df10 = df10.filter(~df10["znak"].isin(list(" \t\n"))) #usuwam niepotrzebne znaki typu  \t\n i spacje
df10 = df10.where(length(df10["znak"]) > 0) #pozostawiam znaki ktore maja dlug wiecej niz 0

df10 = df10.groupBy("nazwa_ksiazki").pivot("znak").agg(count("*").alias("liczba_znakow")) 

df10.show()



end = time.time()
print(f"Uplynelo {end-start} sekund")

+--------------------+----+----+----+----+----+----+----+----+----+----+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+---+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+----+-----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|       nazwa_ksiazki|   !|   "|   #|   $|   %|   &|   '|   (|   )|   *|    ,|  -|   .|   /|   0|   1|   2|   3|   4|   5|   6|   7|   8|   9|  :|   ;|   <|   >|   ?|   @|   [|   ]|   ^|   _|   `|    a|    b|    c|    d|    e|    f|    g|    h|    i|   j|   k|    l|    m|    n|    o|    p|  q|    r|    s|    t|    u|   v|    w|   x|    y|   z|   ||   £|   à|   æ|   ç|   é|   ê|   ï|   œ|   –|   —|   ‘|   ’|   “|   ”|   …|   �|
+--------------------+----+----+----+----+----+----+----+----+----+----+-----+---+----+----+----+----+----+----+----+----+----+----+----+-

In [17]:
x30 = ["Dark-Desire.txt", "Mansfield-Park.txt", "Pride-and-Prejudice.txt", 
       "Sense-and-Sensibility.txt", "The-Secret-of-Poppyridge-Cove.txt", "Alien-Cradle.txt", 
       "From-the-Earth-to-the-Moon.txt", "The-Cat-of-Bubastes.txt", "The-Restless-Sex.txt", 
       "The-Window-at-the-White-Cat.txt","Bones-in-London.txt", "The-Mad-King.txt", 
       "Beyond-The-Rocks.txt", "Simon-Called-Peter.txt", "Smiles.txt","The-Demon-Girl.txt",
       "The-Story-of-Don-Quixote.txt", "Arms-and-the-Woman.txt", "The-Bandbox.txt", "Camille.txt", 
       "Adrift-in-New-York.txt", "An-Amiable-Charlatan-.txt", "Induction.txt", "Tales-of-Terror--Mystery.txt",
       "Tesla.txt", "The-Avenger.txt", "The-Bishop's-Secret.txt", "The-Blonde-Lady.txt", "The-Eye-of-Osiris.txt", 
       "The-Phantom-of-the-Opera.txt"]
url_git30 = "https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/"

ksiazki30 = []
import urllib.request
for i in x30:
    url30 = url_git30 + i
    print(url30)
    with urllib.request.urlopen(url30) as f:
        ksiazka30 = f.read().decode()
        ksiazki30.append(ksiazka30)

https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Dark-Desire.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Mansfield-Park.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Pride-and-Prejudice.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Sense-and-Sensibility.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Secret-of-Poppyridge-Cove.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/Alien-Cradle.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/From-the-Earth-to-the-Moon.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Cat-of-Bubastes.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Restless-Sex.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_Big_Data/main/The-Window-at-the-White-Cat.txt
https://raw.githubusercontent.com/lisaborysenko/Zadanie_

In [21]:
#RDD
#mozemy zauwazyc ze czas wykonania tego zadania za pomoca RDD pobil wszystkie rekordy i trwa srednio od 70 sekund 
start= time.time()
for i, ksiazka30 in enumerate(ksiazki30):
    znaki30 = sc.parallelize(ksiazka30.lower().split())
    liczba30 = znaki30.flatMap(lambda x20: list(x30)).map(lambda x30: (x30, 1)).reduceByKey(lambda x30, y: x30 + y).collect()
    print(f"{x30[i]} {liczba30}")

end = time.time()
print(f"Uplynelo {end-start} sekund") 

Dark-Desire.txt [('Mansfield-Park.txt', 50809), ('Sense-and-Sensibility.txt', 50809), ('The-Secret-of-Poppyridge-Cove.txt', 50809), ('From-the-Earth-to-the-Moon.txt', 50809), ('The-Cat-of-Bubastes.txt', 50809), ('Bones-in-London.txt', 50809), ('Beyond-The-Rocks.txt', 50809), ('Smiles.txt', 50809), ('The-Demon-Girl.txt', 50809), ('Arms-and-the-Woman.txt', 50809), ('Camille.txt', 50809), ('Adrift-in-New-York.txt', 50809), ('An-Amiable-Charlatan-.txt', 50809), ('Induction.txt', 50809), ('Tales-of-Terror--Mystery.txt', 50809), ('Tesla.txt', 50809), ("The-Bishop's-Secret.txt", 50809), ('The-Blonde-Lady.txt', 50809), ('Dark-Desire.txt', 50809), ('Pride-and-Prejudice.txt', 50809), ('Alien-Cradle.txt', 50809), ('The-Restless-Sex.txt', 50809), ('The-Window-at-the-White-Cat.txt', 50809), ('The-Mad-King.txt', 50809), ('Simon-Called-Peter.txt', 50809), ('The-Story-of-Don-Quixote.txt', 50809), ('The-Bandbox.txt', 50809), ('The-Avenger.txt', 50809), ('The-Eye-of-Osiris.txt', 50809), ('The-Phantom-of

In [20]:
#jak mozemy zauwazyc sredni czas wykonania danego zadania z 30 ksiazkami zwiekszyl sie prawie o wiele w porownaniu z 5 ksiazkami az do 23 sekund, ale i tak to jest dobry rezzultat jak na taki duzy zbior ksiazek
start= time.time()


df30 = spark.createDataFrame([(x30[i], ksiazki30[i]) for i in range(len(x30))], ["nazwa_ksiazki", "tekst"])
df30 = df30.withColumn("maly_tekst", lower(df30["tekst"])) 
df30 = df30.withColumn("znak", explode(split(df30["maly_tekst"], ""))) 
#df30 = df30.filter(df30["znak"].isin(list("abcdefghijklmnopqrstuvwxyz"))) 
df30 = df30.filter(~df30["znak"].isin(list(" \t\n\r"))) #usuwam niepotrzebne znaki typu  \t\n i spacje
df30 = df30.where(length(df30["znak"]) > 0) #pozostawiam znaki ktore maja dlug wiecej niz 0

df30 = df30.groupBy("nazwa_ksiazki").pivot("znak").agg(count("*").alias("liczba_znakow")) 

df30.show(30)



end = time.time()
print(f"Uplynelo {end-start} sekund")

+--------------------+----+----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|       nazwa_ksiazki|   !|   "|   #|   $|   %|   &|   '|   (|   )|   *|   +|    ,|   -|   .|   /|   0|   1|   2|   3|   4|   5|   6|   7|   8|   9|  :|   ;|   <|   =|   >|   ?|   @|   [|   ]|   ^|   _|   `|    a|    b|    c|    d|    e|    f|    g|    h|    i|   j|   k|    l|    m|    n|    o|    p|   q|    r|    s|    t|    u|   v|    w|   x|    y|   z|   {|   ||   }|   ~|   £|   ©|   à|   á|   â|   æ|   ç|   è|   é|   ê|   ë|   í|   î|   ï|   ñ|   ô|   ú|   ü|   ă|

Jak widac lepiej korzystac z DF - dataframe ze wzgledu na czas, bo dziala o wiele szybciej aniz RDD.